In [1]:
#IMPORTS

!pip install mediapipe opencv-python

import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mediapipe import solutions as mp_solutions
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

# Unmount the drive if it was previously mounted
!fusermount -u /content/drive
# Remove any existing files in the mount point
!rm -rf /content/drive
#mounting the drive
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 16.1 MB/s eta 0:00:00
fusermount: failed to unmount /content/drive: No such file or directory
Mounted at /content/drive


In [30]:
# Initialize MediaPipe Hands model
mp_hands = mp_solutions.hands
mp_drawing = mp_solutions.drawing_utils

### DEPTH IMAGE DATA

#extract timestamp from filename
def extract_timestamp(filename):
    """Extract timestamp from file name."""
    try:
        parts = filename.split('_')
        if len(parts) > 1:
            return parts[1].split('.')[0]
        return None
    except IndexError:
        return None

#preprocess depth images
def preprocess_depth_image(depth_image, min_depth=10, max_depth=1000):
    """Preprocess depth image by applying filtering and normalization."""
    #normalize depth values to [0, 1]
    depth_image_normalized = cv2.normalize(depth_image, None, alpha=0, beta=1,
                                           norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    #remove noisy values outside the valid depth range --> huge issue with noisy data
    depth_image_filtered = np.where((depth_image > min_depth) & (depth_image < max_depth), depth_image, 0)

    #apply median filtering to smooth noise
    depth_image_smoothed = cv2.medianBlur(depth_image_filtered.astype(np.uint8), ksize=5)

    #fill missing regions using inpainting
    mask = (depth_image_smoothed == 0).astype(np.uint8)
    depth_image_inpainted = cv2.inpaint(depth_image_smoothed, mask, inpaintRadius=5, flags=cv2.INPAINT_TELEA)

    #apply bilateral filtering to refine edges
    depth_image_bilateral = cv2.bilateralFilter(depth_image_inpainted, d=9, sigmaColor=75, sigmaSpace=75)

    return depth_image_bilateral

def match_rgb_and_depth_images(folder_path): #images
    """Match RGB and depth IMAGES by timestamps"""
    file_names = os.listdir(folder_path)
    rgb_files = sorted([f for f in file_names if f.startswith('rgbFrame')])
    depth_files = sorted([f for f in file_names if f.startswith('depthFrame')])

    matched_frames = {}
    for rgb_file in rgb_files:
        timestamp = extract_timestamp(rgb_file)
        if timestamp:
            matched_frames[timestamp] = {'rgb': rgb_file, 'depth': None}

    for depth_file in depth_files:
        timestamp = extract_timestamp(depth_file)
        if timestamp in matched_frames:
            matched_frames[timestamp]['depth'] = depth_file

    return {k: v for k, v in matched_frames.items() if v['rgb'] and v['depth']}



#validate landmarks using depth data
def validate_landmarks(rgb_image, depth_image, hand_landmarks):
    """Validate hand landmarks using depth data."""
    h, w = depth_image.shape
    for landmark in hand_landmarks.landmark:
        cx, cy = int(landmark.x * w), int(landmark.y * h)
        if not (0 <= cx < w and 0 <= cy < h and depth_image[cy, cx] > 0):
            return False
    return True

#compute angles between three points
def compute_angle(a, b, c):
    """Compute the angle formed by three points."""
    ab = np.array(b) - np.array(a)
    bc = np.array(c) - np.array(b)
    cos_theta = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))
    return angle

def compute_vector_angle(a, b, c):
    """
    Compute the angle between vectors ab and bc using the dot product.
    Parameters:
        a, b, c: 3D coordinates of points (list or ndarray).
    """
    ab = np.array(b) - np.array(a)
    bc = np.array(c) - np.array(b)
    cos_theta = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

def compute_abduction_distance(point1, point2):
    """
    Compute the Euclidean distance between two points.
    Parameters:
        point1, point2: 3D coordinates of points.
    """
    return np.linalg.norm(np.array(point1) - np.array(point2))

def process_frames_and_compute_angles(folder_path, matched_frames, finger_indices, finger_colors, use_preprocessed=True):
    """
    Process RGB and depth frames, compute joint angles, and visualize 3D hand poses.
    Uses functions from the referenced paper to calculate flexion and abduction angles.
    """
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)

    for i, (timestamp, files) in enumerate(matched_frames.items()):
        rgb_path = os.path.join(folder_path, files['rgb'])
        depth_path = os.path.join(folder_path, files['depth'])

        # Load RGB and depth images
        rgb_image = cv2.imread(rgb_path)
        depth_image = cv2.imread(depth_path, cv2.IMREAD_GRAYSCALE)

        if use_preprocessed:
            # Apply preprocessing to depth image
            depth_image = preprocess_depth_image(depth_image)

        # Resize RGB to match depth dimensions
        depth_height, depth_width = depth_image.shape
        rgb_image_resized = cv2.resize(rgb_image, (depth_width, depth_height))
        rgb_image_rgb = cv2.cvtColor(rgb_image_resized, cv2.COLOR_BGR2RGB)

        # Process with MediaPipe Hands
        results = hands.process(rgb_image_rgb)
        if not results.multi_hand_landmarks:
            continue

        # Process each detected hand
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks_3d = []
            for landmark in hand_landmarks.landmark:
                cx, cy = int(landmark.x * depth_width), int(landmark.y * depth_height)
                z = depth_image[cy, cx] if (0 <= cx < depth_width and 0 <= cy < depth_height) else 0
                landmarks_3d.append([cx, cy, z])

            # Compute angles for each finger
            for finger, indices in finger_indices.items():
                if len(landmarks_3d) >= max(indices) + 1:
                    # Flexion Angles
                    mcp_angle = compute_vector_angle(landmarks_3d[indices[0]],
                                                     landmarks_3d[indices[1]],
                                                     landmarks_3d[indices[2]])
                    pip_angle = compute_vector_angle(landmarks_3d[indices[1]],
                                                     landmarks_3d[indices[2]],
                                                     landmarks_3d[indices[3]])
                    dip_angle = compute_vector_angle(landmarks_3d[indices[2]],
                                                     landmarks_3d[indices[3]],
                                                     landmarks_3d[indices[4]])

                    # Abduction Distances (between adjacent fingertips)
                    if finger != "thumb":  # Thumb abduction is handled separately
                        prev_tip = landmarks_3d[finger_indices[finger][4] - 4]
                        current_tip = landmarks_3d[indices[4]]
                        abduction_distance = compute_abduction_distance(prev_tip, current_tip)

                        print(f"{finger.capitalize()} Finger Abduction Distance - Frame {i}: {abduction_distance:.2f} mm")

                    print(f"{finger.capitalize()} Finger Angles - Frame {i}:")
                    print(f"  MCP: {mcp_angle:.2f}°")
                    print(f"  PIP: {pip_angle:.2f}°")
                    print(f"  DIP: {dip_angle:.2f}°")

            # Visualize the 3D Hand Pose
            visualize_3d_hand_pose(landmarks_3d, finger_indices, finger_colors, i)

    hands.close()




In [40]:
#RAW DEPTH DATA

#preprocess depth images
def preprocess_depth_image(depth_image, min_depth=10, max_depth=1000, depth_scale=1.0):
    """
    Preprocess depth image by applying filtering and normalization.

    Parameters:
        depth_image (numpy.ndarray): Raw depth image in float32 format.
        min_depth (float): Minimum valid depth value.
        max_depth (float): Maximum valid depth value.
        depth_scale (float): Scale factor to convert raw depth units to millimeters.

    Returns:
        numpy.ndarray: Preprocessed depth image.
    """
    # Convert raw depth to millimeters using the scale factor
    depth_image_scaled = depth_image * depth_scale

    # Remove noisy values outside the valid depth range
    depth_image_filtered = np.where((depth_image_scaled > min_depth) &
                                     (depth_image_scaled < max_depth), depth_image_scaled, 0)

    # Apply median filtering to smooth noise
    depth_image_smoothed = cv2.medianBlur(depth_image_filtered.astype(np.uint16), ksize=5)

    # Create a mask for missing regions
    mask = (depth_image_smoothed == 0).astype(np.uint8)

    # Fill missing regions using inpainting
    depth_image_inpainted = cv2.inpaint(depth_image_smoothed.astype(np.uint8), mask,
                                        inpaintRadius=5, flags=cv2.INPAINT_TELEA)

    # Apply bilateral filtering to refine edges
    depth_image_bilateral = cv2.bilateralFilter(depth_image_inpainted, d=9,
                                                sigmaColor=75, sigmaSpace=75)

    # Normalize depth values to [0, 1] for consistent representation
    depth_image_normalized = cv2.normalize(depth_image_bilateral.astype(np.float32), None,
                                           alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    return depth_image_normalized

def match_rgb_and_depth_files(folder_path): #raw
    """
    Match RGB and depth RAW files by corresponding numbers in filenames.

    Parameters:
        folder_path (str): Path to the folder containing RGB and depth files.

    Returns:
        dict: A dictionary where keys are numbers and values are dictionaries
              with 'rgb' and 'depth' file names.
    """
    def extract_number(file_name):
        """Extract number from file name using regex."""
        match = re.search(r'_(\d+)\.', file_name)  # Matches '_<number>.'
        return int(match.group(1)) if match else None

    # List and sort all files
    file_names = os.listdir(folder_path)
    rgb_files = sorted([f for f in file_names if f.startswith('rgbFrame')])
    depth_files = sorted([f for f in file_names if f.startswith('rawDepth')])

    #print(f"RGB Files: {rgb_files}")
    #print(f"Depth Files: {depth_files}")

    # Create a dictionary to match frames
    matched_frames = {}

    # Populate the dictionary with RGB files
    for rgb_file in rgb_files:
        number = extract_number(rgb_file)
        #print(f"Extracted number for RGB file {rgb_file}: {number}")  # Debugging
        if number is not None:
            matched_frames[number] = {'rgb': rgb_file, 'depth': None}

    # Match depth files to the same numbers
    for depth_file in depth_files:
        number = extract_number(depth_file)
        #print(f"Extracted number for Depth file {depth_file}: {number}")  # Debugging
        if number is not None and number in matched_frames:
            matched_frames[number]['depth'] = depth_file

    # Filter unmatched pairs
    matched_frames = {k: v for k, v in matched_frames.items() if v['depth']}
    print(f"Matched Frames: {matched_frames}")  # Debugging

    return matched_frames

In [41]:
# Main Execution
folder_path = "/content/drive/MyDrive/cs-283-assignments/Final Project/hand_raw"
finger_indices = {
    "thumb": [0, 1, 2, 3, 4],
    "index": [0, 5, 6, 7, 8],
    "middle": [0, 9, 10, 11, 12],
    "ring": [0, 13, 14, 15, 16],
    "pinky": [0, 17, 18, 19, 20],
}
finger_colors = {
    "thumb": "red",
    "index": "green",
    "middle": "blue",
    "ring": "orange",
    "pinky": "purple",
}

matched_frames = match_rgb_and_depth_files(folder_path)

Matched Frames: {0: {'rgb': 'rgbFrame_0.png', 'depth': 'rawDepth_0.raw'}, 1: {'rgb': 'rgbFrame_1.png', 'depth': 'rawDepth_1.raw'}, 10: {'rgb': 'rgbFrame_10.png', 'depth': 'rawDepth_10.raw'}, 11: {'rgb': 'rgbFrame_11.png', 'depth': 'rawDepth_11.raw'}, 12: {'rgb': 'rgbFrame_12.png', 'depth': 'rawDepth_12.raw'}, 13: {'rgb': 'rgbFrame_13.png', 'depth': 'rawDepth_13.raw'}, 14: {'rgb': 'rgbFrame_14.png', 'depth': 'rawDepth_14.raw'}, 15: {'rgb': 'rgbFrame_15.png', 'depth': 'rawDepth_15.raw'}, 16: {'rgb': 'rgbFrame_16.png', 'depth': 'rawDepth_16.raw'}, 17: {'rgb': 'rgbFrame_17.png', 'depth': 'rawDepth_17.raw'}, 18: {'rgb': 'rgbFrame_18.png', 'depth': 'rawDepth_18.raw'}, 19: {'rgb': 'rgbFrame_19.png', 'depth': 'rawDepth_19.raw'}, 2: {'rgb': 'rgbFrame_2.png', 'depth': 'rawDepth_2.raw'}, 20: {'rgb': 'rgbFrame_20.png', 'depth': 'rawDepth_20.raw'}, 21: {'rgb': 'rgbFrame_21.png', 'depth': 'rawDepth_21.raw'}, 22: {'rgb': 'rgbFrame_22.png', 'depth': 'rawDepth_22.raw'}, 23: {'rgb': 'rgbFrame_23.png', '

In [56]:
def visualize_3d_hand_pose_with_rgb(landmarks_3d, rgb_frame, frame_index):
    """
    Visualize 3D hand pose alongside the corresponding RGB frame using matplotlib.

    Parameters:
        landmarks_3d (list): List of 3D landmarks as [x, y, z].
        rgb_frame (numpy.ndarray): RGB frame as a numpy array (from cv2.imread or similar).
        frame_index (int): Current frame index for visualization.
    """
    import matplotlib.pyplot as plt

    # Ensure RGB frame is valid
    if rgb_frame is None or rgb_frame.size == 0:
        print("Error: RGB frame is empty or invalid.")
        return

    # External definitions of finger_indices and finger_colors
    global finger_indices, finger_colors

    fig = plt.figure(figsize=(12, 6))

    # Plot the RGB frame
    ax1 = fig.add_subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
    try:
        ax1.imshow(cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for display
        ax1.set_title(f"RGB Frame - Frame {frame_index}")
        ax1.axis('off')  # Hide axis for the image
    except Exception as e:
        print(f"Error displaying RGB frame: {e}")
        return

    # Plot the 3D hand pose
    ax2 = fig.add_subplot(1, 2, 2, projection='3d')  # 1 row, 2 columns, 2nd subplot
    try:
        xs = [pt[0] for pt in landmarks_3d]
        ys = [pt[1] for pt in landmarks_3d]
        zs = [pt[2] for pt in landmarks_3d]

        # Plot all landmarks
        ax2.scatter(xs, ys, zs, c='black', s=20, label='Landmarks')

        # Connect landmarks for each finger
        for finger, indices in finger_indices.items():
            finger_points = [landmarks_3d[i] for i in indices]
            x_finger = [pt[0] for pt in finger_points]
            y_finger = [pt[1] for pt in finger_points]
            z_finger = [pt[2] for pt in finger_points]

            # Use predefined color for the finger
            color = finger_colors[finger]
            ax2.plot(x_finger, y_finger, z_finger, label=finger.capitalize(), color=color)

        # Set axis labels and title
        ax2.set_xlabel('X (pixels)')
        ax2.set_ylabel('Y (pixels)')
        ax2.set_zlabel('Z (depth)')
        ax2.set_title(f"3D Hand Pose - Frame {frame_index}")
        ax2.legend()
    except Exception as e:
        print(f"Error displaying 3D plot: {e}")
        return

    # Display both subplots
    plt.tight_layout()
    plt.show()





def process_frames_and_debug_compute_angles(folder_path, matched_frames, finger_indices, finger_colors, use_preprocessed=True):
    """
    Process RGB and depth frames, debug raw depth data loading, compute joint angles,
    and visualize 3D hand poses.
    """
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)

    for i, (timestamp, files) in enumerate(matched_frames.items()):
        rgb_path = os.path.join(folder_path, files['rgb'])
        depth_path = os.path.join(folder_path, files['depth'])

        print(f"\nProcessing Frame {i}: Timestamp {timestamp}")
        print(f"RGB Path: {rgb_path}")
        print(f"Depth Path: {depth_path}")

        # Load RGB and depth images
        rgb_image = cv2.imread(rgb_path)
        if rgb_image is None:
            print(f"Error: Failed to load RGB image at {rgb_path}")
            continue

        try:
            with open(depth_path, 'rb') as f:
                depth_image = np.fromfile(f, dtype=np.float32)
            depth_image = depth_image.reshape((480, 640))  # Replace with actual dimensions
        except Exception as e:
            print(f"Error loading depth file {depth_path}: {e}")
            continue

        if use_preprocessed:
            # Apply preprocessing to depth image
            try:
                depth_image = preprocess_depth_image(depth_image)
            except Exception as e:
                print(f"Error preprocessing depth image: {e}")
                continue

        # Resize RGB to match depth dimensions
        depth_height, depth_width = depth_image.shape
        try:
            rgb_image_resized = cv2.resize(rgb_image, (depth_width, depth_height))
            rgb_image_rgb = cv2.cvtColor(rgb_image_resized, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(f"Error resizing or converting RGB image: {e}")
            continue

        # Process with MediaPipe Hands
        results = hands.process(rgb_image_rgb)
        if not results.multi_hand_landmarks:
            print("No hands detected in this frame.")
            continue

        # Process each detected hand
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks_3d = []
            for landmark in hand_landmarks.landmark:
                cx, cy = int(landmark.x * depth_width), int(landmark.y * depth_height)
                z = depth_image[cy, cx] if (0 <= cx < depth_width and 0 <= cy < depth_height) else 0
                landmarks_3d.append([cx, cy, z])

            # Compute angles for each finger
            for finger, indices in finger_indices.items():
                if len(landmarks_3d) >= max(indices) + 1:
                    try:
                        # Flexion Angles
                        mcp_angle = compute_vector_angle(landmarks_3d[indices[0]],
                                                         landmarks_3d[indices[1]],
                                                         landmarks_3d[indices[2]])
                        pip_angle = compute_vector_angle(landmarks_3d[indices[1]],
                                                         landmarks_3d[indices[2]],
                                                         landmarks_3d[indices[3]])
                        dip_angle = compute_vector_angle(landmarks_3d[indices[2]],
                                                         landmarks_3d[indices[3]],
                                                         landmarks_3d[indices[4]])

                        print(f"{finger.capitalize()} Finger Angles - Frame {i}:")
                        print(f"  MCP: {mcp_angle:.2f}°")
                        print(f"  PIP: {pip_angle:.2f}°")
                        print(f"  DIP: {dip_angle:.2f}°")

                        # Abduction Distances
                        if finger != "thumb":  # Thumb abduction is handled separately
                            prev_tip = landmarks_3d[finger_indices[finger][4] - 4]
                            current_tip = landmarks_3d[indices[4]]
                            abduction_distance = compute_abduction_distance(prev_tip, current_tip)
                            print(f"{finger.capitalize()} Finger Abduction Distance - Frame {i}: {abduction_distance:.2f} mm")
                    except Exception as e:
                        print(f"Error computing angles or distances for {finger}: {e}")

            # Visualize the 3D Hand Pose
            try:
                visualize_3d_hand_pose(landmarks_3d, rgb_image_rgb, i)
            except Exception as e:
                print(f"Error visualizing 3D hand pose: {e}")

    hands.close()


In [43]:
process_frames_and_debug_compute_angles(folder_path, matched_frames, finger_indices, finger_colors, use_preprocessed=True)

Output hidden; open in https://colab.research.google.com to view.

In [57]:
#Participant 1
participant_1_path = '/content/drive/MyDrive/cs-283-assignments/Final Project/hand_participant'
process_frames_and_debug_compute_angles(participant_1_path, matched_frames, finger_indices, finger_colors, use_preprocessed=True)

Output hidden; open in https://colab.research.google.com to view.

## 4 Poses Classifier

In [58]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

file_path = "/content/drive/MyDrive/cs-283-assignments/Final Project/Pose_Dataset.csv"
df = pd.read_csv(file_path)
df.head()


,Frame,Thumb MCP,Thumb PIP,Thumb DIP,Index MCP,Index PIP,Index DIP,Index Abduction,Middle MCP,Middle PIP,...,Middle Abduction,Ring MCP,Ring PIP,Ring DIP,Ring Abduction,Pinky MCP,Pinky PIP,Pinky DIP,Pinky Abduction,Pose
0,6,24.73,0.32,8.81,1.31,0.66,1.44,118.53,7.15,0.24,...,65.92,5.36,2.68,0.42,28.84,4.66,0.16,2.53,83.63,Pose 1
1,7,25.11,0.64,11.50,2.49,1.68,2.69,118.12,8.29,0.36,...,64.47,4.42,2.46,0.32,29.15,5.52,0.80,1.89,83.55,Pose 1
2,12,26.15,1.17,8.81,1.46,1.80,4.39,122.67,8.22,0.85,...,62.30,3.18,2.94,1.38,29.68,4.66,1.55,5.08,80.72,Pose 1
3,16,34.53,7.16,9.10,2.88,2.14,2.22,98.90,1.80,0.53,...,46.52,7.76,3.43,0.00,32.65,0.11,2.05,1.01,79.40,Pose 1
4,10,38.19,24.31,11.50,14.86,5.19,2.90,86.41,3.54,0.00,...,24.04,11.36,2.30,0.77,24.08,26.88,4.40,1.91,44.42,Pose 2


In [60]:
#normalize features (exclude 'Frame' and 'Pose')
scaler = MinMaxScaler()
feature_columns = df.columns[1:-1]  # All columns except 'Frame' and 'Pose'
df[feature_columns] = scaler.fit_transform(df[feature_columns])

def augment_data(df, num_augmentations=10, angle_variation=0.05):
    augmented_data = []

    for _, row in df.iterrows():
        base_features = row[feature_columns].values
        label = row["Pose"]

        for _ in range(num_augmentations):
            # Add random noise to features
            augmented_features = base_features + np.random.uniform(
                -angle_variation, angle_variation, size=base_features.shape
            )
            # Clamp values to [0, 1] range
            augmented_features = np.clip(augmented_features, 0, 1)
            augmented_data.append(np.append(augmented_features, label))

    augmented_columns = feature_columns.tolist() + ["Pose"]
    augmented_df = pd.DataFrame(augmented_data, columns=augmented_columns)
    return augmented_df

#augment the dataset
augmented_df = augment_data(df, num_augmentations=10)
print("Augmented Dataset:")
print(augmented_df.head())


Augmented Dataset:
   Thumb MCP  Thumb PIP  Thumb DIP  Index MCP  Index PIP  Index DIP  \
0   0.004839   0.000000   0.728602   0.014265   0.020271   0.000000   
1   0.001191   0.000000   0.705886   0.000000   0.000000   0.012805   
2   0.000000   0.000000   0.706351   0.000453   0.036048   0.004728   
3   0.048822   0.000000   0.679012   0.041850   0.000000   0.000000   
4   0.022434   0.037997   0.689007   0.049638   0.000000   0.043351   

   Index Abduction  Middle MCP  Middle PIP  Middle DIP  Middle Abduction  \
0         0.938172    0.051369    0.048547     0.00000          0.991247   
1         0.960543    0.073758    0.008147     0.04929          0.952921   
2         0.985342    0.004466    0.000000     0.00000          0.958856   
3         0.993318    0.094775    0.008551     0.00816          1.000000   
4         0.984982    0.033486    0.000000     0.00000          0.975475   

   Ring MCP  Ring PIP  Ring DIP  Ring Abduction  Pinky MCP  Pinky PIP  \
0  0.054117  0.000000  0

In [62]:
# need more data ---> but can now use this for real time app pose classification
#to make sure users are doing correct poses so we can properly track progress

# split the dataset into features and labels
X = augmented_df[feature_columns].values
y = augmented_df["Pose"].values

#train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#train Random Forest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

#evaluate the classifier ---> to use once i add more data
y_pred = clf.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

      Pose 1       1.00      1.00      1.00         2
      Pose 3       1.00      1.00      1.00         2
      Pose 4       1.00      1.00      1.00         2

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

Confusion Matrix:
[[2 0 0]
 [0 2 0]
 [0 0 2]]


In [63]:
from zipfile import ZipFile
%cd "/content/drive/My Drive/cs-283-assignments/Final Project"

def writeFolder(zip, path):
  import os
  for dirpath,dirs,files in os.walk(path):
    for f in files:
      fn = os.path.join(dirpath, f)
      zip.write(fn)

def writeFile(zip, path):
  zip.write(path)

/content/drive/My Drive/cs-283-assignments/Final Project


In [64]:
# Zip source
zipObj = ZipFile('REBEI_RIMA_final_project.zip', 'w')

writeFolder(zipObj, "CV_Final_Project")
writeFile(zipObj, "CV_Final_Project.ipynb")

zipObj.close()

In [65]:
!jupyter nbconvert --to html CV_Final_Project.ipynb --output "REBEI_RIMA_final_project.html"

[NbConvertApp] Converting notebook CV_Final_Project.ipynb to html
[NbConvertApp] Writing 391793 bytes to REBEI_RIMA_final_project.html
